In [1]:
import torch
from torch import nn

import pandas as pd
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
from sklearn.datasets import make_circles


# Make 1000 samples 
n_samples = 1000

# Create circles
X, y = make_circles(n_samples,
                    noise=0.03, # a little bit of noise to the dots
                    random_state=42) # keep random state so we get the same values

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [40]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.Linear(10, 1),
            
        )
        
    def forward(self, x):
        return self.linear_relu_stack(x)
    
model = BinaryClassifier().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(params=model.parameters(), lr=0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

torch.manual_seed(42)
epochs = 2000
for epoch in range(epochs):
    model.train()
    
    # Forward Pass
    y_train_logits = model(X_train).squeeze()
    y_train_preds = torch.round(torch.sigmoid(y_train_logits))
    
    # Calculate loss
    train_loss = loss_fn(y_train_logits, y_train)
    
    optim.zero_grad()
    
    train_loss.backward()
    
    optim.step()
    
    model.eval()
    with torch.inference_mode():
        y_test_logits = model(X_test).squeeze()
        y_test_preds = torch.round(torch.sigmoid(y_test_logits))
        test_loss = loss_fn(y_test_logits, y_test)
        test_acc = accuracy_score(y_test_preds, y_test)
        
    if epoch % 100 == 0:
        print(f'test loss: {test_loss}, test acc: {test_acc}')

test loss: 0.6907749772071838, test acc: 0.516
test loss: 0.6887218356132507, test acc: 0.556
test loss: 0.6858190298080444, test acc: 0.58
test loss: 0.6819847822189331, test acc: 0.568
test loss: 0.6766068339347839, test acc: 0.572
test loss: 0.668337881565094, test acc: 0.58
test loss: 0.655248761177063, test acc: 0.652
test loss: 0.6331680417060852, test acc: 0.74
test loss: 0.5904598832130432, test acc: 0.824
test loss: 0.5181279182434082, test acc: 0.952
test loss: 0.4095374643802643, test acc: 0.984
test loss: 0.2861171066761017, test acc: 1.0
test loss: 0.19242051243782043, test acc: 1.0
test loss: 0.13653047382831573, test acc: 1.0
test loss: 0.10375943779945374, test acc: 1.0
test loss: 0.08339163661003113, test acc: 1.0
test loss: 0.0698368027806282, test acc: 1.0
test loss: 0.060230229049921036, test acc: 1.0
test loss: 0.053121186792850494, test acc: 1.0
test loss: 0.04767268896102905, test acc: 1.0
